In [41]:
using Convex
using PyPlot

In [48]:
# Generate random indefinite QP
n = 10
X = randn(n, n)
A = Symmetric(X)
lambda= eigvals(A)
# plt[:hist](lambda)
b = randn(n)

10-element Array{Float64,1}:
 -1.22375  
 -0.285587 
 -0.858333 
 -1.29053  
  1.27454  
 -0.713199 
  0.218869 
  0.498342 
 -0.182068 
  0.0246196

In [63]:
x = Variable(n)
X = Variable(n, n)
constraints = [
    trace(X) <= 3,
    X == X',
    lambdamin([X x; x' 1]) > 0
]
problem = minimize(trace(A*X) + 2*b'*x, constraints)
solve!(problem)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int64}, ::Int64) at ./deprecated.jl:57
 [3] conic_form!(::Convex.CTransposeAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/transpose.jl:114
 [4] conic_form!(::Convex.NegateAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/add_subtract.jl:47
 [5] conic_form!(::Convex.AdditionAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/add_subtract.jl:108
 [6] conic_form!(::Convex.EqConstraint, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/constraints/constraints.jl:43
 [7] conic_form!(::Convex.Problem, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/problems.jl:101
 [8] conic_problem(::Convex.Problem) at /Users/mark/.julia/v0.6/Convex/src/problems.jl:124
 [9] #solve!#25(::Bool, ::Bool, ::Bool, ::Function, ::Convex.Problem) at /Users/mark/.julia/v0.6/Convex/src/solution.jl:25
 

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Int64}, ::Int64) at ./deprecated.jl:57
 [3] conic_form!(::Convex.TransposeAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/transpose.jl:51
 [4] conic_form!(::Convex.HcatAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/stack.jl:47
 [5] conic_form!(::Convex.TransposeAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/transpose.jl:44
 [6] conic_form!(::Convex.AdditionAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/add_subtract.jl:108
 [7] conic_form!(::Convex.DotMultiplyAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/multiply_divide.jl:175
 [8] conic_form!(::Convex.IndexAtom, ::Convex.UniqueConicForms) at /Users/mark/.julia/v0.6/Convex/src/atoms/affine/index.jl:72
 [9] conic_form!(::Convex.SDPConstraint, ::Convex.UniqueConicForms) at 

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 468
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 112, constraints m = 224
Cones:	primal zero / dual free vars: 156
	linear vars: 2
	sd vars: 66, sd blks: 1
Setup time: 4.83e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf      -inf       inf  2.57e-03 
   100| 1.33e-03  1.79e-01  9.79e-04 -1.93e+01 -1.92e+01  7.98e-17  9.36e-03 
   200| 1.75e-03  2.28e-01  2.22e-03 -1.96e+01 -1.95e+01  4.88e-16  1.82e-02 
   300| 5.09e-05  3.47e-03  2

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] isnan(::Array{Float64,1}) at ./deprecated.jl:57
 [3] populate_solution!(::SCS.SCSMathProgModel, ::Convex.Problem, ::Dict{UInt64,Tuple{Int64,Int64}}, ::Array{Convex.ConicConstr,1}) at /Users/mark/.julia/v0.6/Convex/src/solution.jl:118
 [4] #solve!#25(::Bool, ::Bool, ::Bool, ::Function, ::Convex.Problem) at /Users/mark/.julia/v0.6/Convex/src/solution.jl:38
 [5] solve!(::Convex.Problem) at /Users/mark/.julia/v0.6/Convex/src/solution.jl:21
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/mark/.julia/v0.6/IJulia/src/execute_request.jl:158
 [8] (::Compat.#inner#17{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /Users/mark/.julia/v0.6/Compat/src/Compat.jl:385
 [9] eventloop(::ZMQ.Socket) at /Users/mark/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[63], in expression starting 

In [64]:
1/2

0.5